In [1]:
%matplotlib inline

from modsim import *

In [ ]:
condition = Condition(y=0, )

In [2]:
def slope_func(state,t,system):
    m, y, vy = state
    unpack(system)
    m_dot = rho * A_surf * B_rate
    ay = (v_exhaust * m_dot) / m - g - F_drag / m
    return vy, ay